# Colab Setup

In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

drive_path = <Your drive PATH>
caption_dir = drive_path + "modified_captions/"

Mounted at /content/drive


In [2]:
!pip install textattack
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 14.8 MB/s 
     |████████████████████████████████| 60 kB 9.7 MB/s 
     |████████████████████████████████| 769 kB 63.2 MB/s 
     |████████████████████████████████| 125 kB 55.7 MB/s 
     |████████████████████████████████| 5.5 MB 59.6 MB/s 
     |████████████████████████████████| 3.6 MB 56.4 MB/s 
     |████████████████████████████████| 41.4 MB 1.2 MB/s 
     |████████████████████████████████| 401 kB 67.2 MB/s 
     |████████████████████████████████| 365 kB 68.2 MB/s 
     |████████████████████████████████| 115 kB 74.9 MB/s 
     |████████████████████████████████| 163 kB 77.9 MB/s 
     |████████████████████████████████| 95 kB 6.3 MB/s 
     |████████████████████████████████| 212 kB 70.6 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
     |████████████████████████████████| 7.6 MB 59.1 MB/s 
     |█████████████████████████████

In [3]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
import os

# TextAttack

In [7]:
captions_df = pd.read_csv(drive_path + "processed_data/coco_objs_attrs_captions.csv", index_col=0)
print(captions_df.shape)
captions_df.sample(10)

(25014, 6)


,image_id,file,url,objects,attributes,original
1084,"216 578545\nName: id, dtype: int64",000000578545.jpg,216 http://images.cocodataset.org/val2017/0...,"['wall', 'bed', 'woman', 'shirt', 'wall', 'wat...","['white', 'blue', 'smiling', 'white', 'white',...",A pregnant woman laying in bed reading a book
20263,"4050 52413\nName: id, dtype: int64",000000052413.jpg,4050 http://images.cocodataset.org/val2017/...,"['person', 'hand', 'shoe', 'phone', 'jeans', '...","['sitting', 'white', 'white', 'pink', 'blue', ...",there is a adult that is holding a pink prince...
9083,"1814 474164\nName: id, dtype: int64",000000474164.jpg,1814 http://images.cocodataset.org/val2017/...,"['sky', 'dog', 'car', 'truck', 'tree', 'buildi...","['blue', 'black', 'silver', 'silver', 'green',...",a little dog sitting on the roof of a club cab...
20892,"4176 95069\nName: id, dtype: int64",000000095069.jpg,4176 http://images.cocodataset.org/val2017/...,"['building', 'sky', 'tower', 'building', 'towe...","['old', 'blue', 'tall', 'green', 'pointed', 'l...",A european city in nice a sunny bright day
14467,"2891 244181\nName: id, dtype: int64",000000244181.jpg,2891 http://images.cocodataset.org/val2017/...,"['table', 'sandwich', 'bottle', 'table', 'brea...","['blue', 'brown', 'white', 'blue', 'brown', 't...",Here are two kinds of sandwiches and a pie wi...
12428,"2483 3501\nName: id, dtype: int64",000000003501.jpg,2483 http://images.cocodataset.org/val2017/...,"['plate', 'plate', 'broccoli', 'rice', 'brocco...","['white', 'white', 'green', 'white', 'green', ...","Looking down at a bowl of white rice, broccoli..."
9068,"1811 298697\nName: id, dtype: int64",000000298697.jpg,1811 http://images.cocodataset.org/val2017/...,"['field', 'sky', 'grass', 'animals', 'cloud', ...","['brown', 'blue', 'brown', 'grazing', 'white',...",Two horses in a pasture with buildings in the ...
11159,"2229 388903\nName: id, dtype: int64",000000388903.jpg,2229 http://images.cocodataset.org/val2017/...,"['man', 'building', 'woman', 'hat', 'bag', 'ha...","['standing', 'white', 'standing', 'red', 'yell...",A woman is on a phone in front of a fruit stand.
20292,"4056 279278\nName: id, dtype: int64",000000279278.jpg,4056 http://images.cocodataset.org/val2017/...,"['shirt', 'people', 'dog', 'tire', 'person', '...","['white', 'standing', 'white', 'black', 'stand...",a person riding a skate board on a city street
23995,"4796 292456\nName: id, dtype: int64",000000292456.jpg,4796 http://images.cocodataset.org/val2017/...,"['building', 'door', 'shirt', 'man', 'tree', '...","['white', 'open', 'brown', 'standing', 'small'...",a woman is sitting on a scooter outside


In [ ]:
og_caps = list(captions_df['original'])

### Easy Data Augmenter

In [ ]:
from textattack.augmentation import EasyDataAugmenter

augmenter = EasyDataAugmenter(pct_words_to_swap=0.2, transformations_per_example=1)
eda_aug_caps = augmenter.augment_many(og_caps)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
eda_aug_caps[:10]

[['A man is in a kitchen fix pizzas.'],
 ['Man in apron standing on front of oven with humankind pans and bakeware'],
 ['vagabond A baker is working in the kitchen rolling dough.'],
 ['A individual standing by a stove in a kitchen.'],
 ['A hanging with near being made and a person standing pies a wall with pots and pans table on the wall.'],
 ['din The dining table near the kitchen has a bowl of fruit on it.'],
 ['A small kitchen has various appliances and diverse a table.'],
 ['The kitchen is sporty and ready for us to hear.'],
 ['A kitchen and in area decorated dining white.'],
 ['A kitchen that has a table of fruit on the bowl.']]

In [ ]:
eda_aug = captions_df
eda_aug['modified'] = [c[0] for c in eda_aug_caps]

In [ ]:
eda_aug.sample(5)

,image_id,file,url,objects,attributes,original,modified
4727,"944 125936\nName: id, dtype: int64",000000125936.jpg,944 http://images.cocodataset.org/val2017/0...,"['man', 'man', 'people', 'shirt', 'bananas', '...","['standing', 'white', 'eating', 'plaid', 'yell...",Two customers and three employees interact in ...,Two customers and three employees interact in ...
19734,"3944 96549\nName: id, dtype: int64",000000096549.jpg,3944 http://images.cocodataset.org/val2017/...,"['plane', 'runway', 'tail', 'wheels', 'light',...","['white', 'black', 'blue', 'black', 'red', 'wh...",A commercial plane on the strip to take off.,A commercial plane on the strip to take necess...
2917,"583 91406\nName: id, dtype: int64",000000091406.jpg,583 http://images.cocodataset.org/val2017/0...,"['boxes', 'women', 'room', 'wall', 'wall', 'wo...","['cardboard', 'standing', 'white', 'white', 'w...",Four women stand behind boxes of large pizzas.,Four women behind boxes of large pizzas.
9409,"1879 302030\nName: id, dtype: int64",000000302030.jpg,1879 http://images.cocodataset.org/val2017/...,"['bed', 'keyboard', 'desk', 'computer', 'scree...","['white', 'white', 'white', 'white', 'white', ...",The desk with a laptop has papers strewn all o...,The desk with a IT laptop has papers strewn al...
10132,"2024 474344\nName: id, dtype: int64",000000474344.jpg,2024 http://images.cocodataset.org/val2017/...,"['woman', 'grass', 'man', 'shirt', 'bat', 'bui...","['standing', 'green', 'young', 'white', 'green...",A man holding a baseball bat at a home plate.,A gentleman defy a baseball bat at a home plate.


In [ ]:
eda_aug = eda_aug.drop(["file", "url", "objects", "attributes"], axis=1)

In [ ]:
eda_aug.shape

(25014, 3)

In [ ]:
eda_aug.to_csv(caption_dir + "MSCOCO/coco_eda_modified.csv", index=False)

### Deletion Augmenter

In [ ]:
from textattack.augmentation import DeletionAugmenter

augmenter = DeletionAugmenter(transformations_per_example=1)
del_aug_caps = augmenter.augment_many(og_caps)

In [ ]:
del_aug_caps[:10]

[['A man is in a kitchen making.'],
 ['Man apron standing on front of oven with pans and bakeware'],
 ['A baker is in the kitchen rolling dough.'],
 ['A person standing by a stove in kitchen.'],
 ['A table with pies made and a person standing near a wall with pots and pans hanging on wall.'],
 ['The dining table near the kitchen has a bowl fruit on it.'],
 ['A small kitchen has various appliances and a.'],
 ['The is clean and ready for us to see.'],
 ['A kitchen and dining area decorated in.'],
 ['A kitchen that has bowl of fruit on the table.']]

In [ ]:
del_aug = captions_df
del_aug['modified'] = [c[0] for c in del_aug_caps]

In [ ]:
del_aug.sample(5)

,image_id,file,url,objects,attributes,original,modified
24143,"4825 14226\nName: id, dtype: int64",000000014226.jpg,4825 http://images.cocodataset.org/val2017/...,"['man', 'window', 'table', 'laptop', 'laptop',...","['sitting', 'silver', 'white', 'silver', 'gray...",A man in a green suiit leaning over a laptop,A in a green suiit leaning over a laptop
24419,"4881 326128\nName: id, dtype: int64",000000326128.jpg,4881 http://images.cocodataset.org/val2017/...,"['snow', 'man', 'sky', 'hat', 'snowboard', 'mo...","['white', 'white', 'blue', 'black', 'black', '...",The man is posing for a picture with his snowb...,The man is posing a picture with his snowboard.
18626,"3723 144706\nName: id, dtype: int64",000000144706.jpg,3723 http://images.cocodataset.org/val2017/...,"['street', 'car', 'sign', 'light', 'building',...","['wet', 'gray', 'green', 'black', 'white', 'me...",A picture of a street light through a rainy lens.,picture of a street light through a rainy lens.
2253,"450 518770\nName: id, dtype: int64",000000518770.jpg,450 http://images.cocodataset.org/val2017/0...,"['room', 'carpet', 'shirt', 'pants', 'boy', 'b...","['gray', 'gray', 'pink', 'gray', 'young', 'you...",People playing the Wii together in a room.,People playing the Wii in a room.
15456,"3089 147338\nName: id, dtype: int64",000000147338.jpg,3089 http://images.cocodataset.org/val2017/...,"['fire hydrant', 'sidewalk', 'tire', 'tire', '...","['yellow', 'gray', 'black', 'black', 'wooden',...",A fire hydrant that has been painted green on ...,A fire hydrant that has been green on top.


In [ ]:
del_aug = del_aug.drop(["file", "url", "objects", "attributes"], axis=1)

In [ ]:
del_aug.shape

(25014, 3)

In [ ]:
del_aug.to_csv(caption_dir + "MSCOCO/coco_del_modified.csv", index=False)

### CLARE Augmenter

In [ ]:
# May require a restart to work, long compute time (can be parallelized by breaking into chunks)
from textattack.augmentation.recipes import CLAREAugmenter

augmenter = CLAREAugmenter(transformations_per_example=1, fast_augment=True)
clare_aug_caps = augmenter.augment_many(og_caps, show_progress=True)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Augmenting data...:   0%|          | 0/12507 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:591: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/75.2M [00:00<?, ?B/s]

2022-10-28 19:36:32,191 loading file /root/.flair/models/upos-english-fast/b631371788604e95f27b6567fe7220e4a7e8d03201f3d862e6204dbf90f9f164.0afb95b43b32509bf4fcc3687f7c64157d8880d08f813124c1bd371c3d8ee3f7
2022-10-28 19:36:32,274 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, INTJ, PUNCT, VERB, PRON, NOUN, ADV, DET, ADJ, ADP, NUM, PROPN, CCONJ, PART, AUX, X, SYM, <START>, <STOP>


Augmenting data...: 100%|██████████| 12507/12507 [10:01:31<00:00,  2.89s/it]


In [ ]:
clare_aug_caps[:10]

[['A man is in a kitchen making potato pizzas.'],
 ['Pan in apron standing on front of oven with pans and bakeware'],
 ['A baker is working in the Starbucks kitchen rolling dough.'],
 ['A person standing by a fridge in a kitchen.'],
 ['A table with pies being made and a person standing near a wall with tin pots and pans hanging on the wall.'],
 ['The dining table near the kitchen has a bowl stuffed of fruit on it.'],
 ['A small kitchen has contained various appliances and a table.'],
 ['The kitchen is clean clean and ready for us to see.'],
 ['A secret kitchen and dining area decorated in white.'],
 ['A kitchen that has a bowl up of fruit on the table.']]

In [ ]:
clare_aug = captions_df
clare_aug['modified'] = [c[0] for c in clare_aug_caps] + og_caps[n_2:]

In [ ]:
clare_aug.sample(5)

,image_id,file,url,objects,attributes,original,modified
20572,"4112 170739\nName: id, dtype: int64",000000170739.jpg,4112 http://images.cocodataset.org/val2017/...,"['elephants', 'elephant', 'ground', 'trunk', '...","['red', 'gray', 'brown', 'long', 'long', 'long...",a couple of elephants are standing behind a fence,a couple of elephants are standing behind a fence
3497,"698 567640\nName: id, dtype: int64",000000567640.jpg,698 http://images.cocodataset.org/val2017/0...,"['grass', 'grass', 'ball', 'shorts', 'ball', '...","['green', 'green', 'white', 'blue', 'white', '...",Group of soccer players on field kicking ball.,Group of three on field kicking ball.
6824,"1363 557916\nName: id, dtype: int64",000000557916.jpg,1363 http://images.cocodataset.org/val2017/...,"['hair', 'bed', 'sock', 'wall', 'shorts', 'bea...","['blonde', 'white', 'white', 'white', 'green',...",A girl sitting on a bed in a room.,A girl sitting on a bed in underneath a room.
7819,"1562 270386\nName: id, dtype: int64",000000270386.jpg,1562 http://images.cocodataset.org/val2017/...,"['window', 'signs', 'sign', 'sign', 'arrow', '...","['glass', 'blue', 'blue', 'blue', 'white', 'ta...",A one way sign and two other signs mounted to ...,A one way sign and two other signs mounted to ...
18879,"3773 192670\nName: id, dtype: int64",000000192670.jpg,3773 http://images.cocodataset.org/val2017/...,"['fence', 'boy', 'helmet', 'glove', 'tree', 'h...","['chain link', 'playing', 'blue', 'brown', 'gr...","A male child swinging his bat at a ball, anoth...","A male child swinging his bat at a ball, anoth..."


In [ ]:
clare_aug = clare_aug.drop(["file", "url", "objects", "attributes"], axis=1)

In [ ]:
clare_aug.shape

(25014, 3)

In [ ]:
clare_aug.to_csv(caption_dir + "MSCOCO/coco_clare_modified.csv", index=False)

### Checklist Attack

In [ ]:
from textattack.augmentation import CheckListAugmenter

augmenter = CheckListAugmenter()
check_aug_caps = augmenter.augment_many(og_caps)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:591: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-10-28 18:36:19,986 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-10-28 18:36:22,455 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
check_aug_caps[:10]

[['A man is in a kitchen making pizzas.'],
 ['Man in apron standing on front of oven with pans and bakeware'],
 ['A baker is working in the kitchen rolling dough.'],
 ['A person standing by a stove in a kitchen.'],
 ['A table with pies being made and a person standing near a wall with pots and pans hanging on the wall.'],
 ['The dining table near the kitchen has a bowl of fruit on it.'],
 ['A small kitchen has various appliances and a table.'],
 ['The kitchen is clean and ready for us to see.'],
 ['A kitchen and dining area decorated in white.'],
 ['A kitchen that has a bowl of fruit on the table.']]

In [ ]:
check_aug = captions_df
check_aug['modified'] = [c[0] for c in check_aug_caps]

In [ ]:
check_aug.sample(5)

,image_id,file,url,objects,attributes,original,modified
12946,"2587 4795\nName: id, dtype: int64",000000004795.jpg,2587 http://images.cocodataset.org/val2017/...,"['cat', 'laptop', 'desk', 'screen', 'laptop', ...","['gray', 'white', 'gray', 'gray', 'silver', 'w...",A cat is sitting and watching a computer screen.,A cat is sitting and watching a computer screen.
300,"60 360661\nName: id, dtype: int64",000000360661.jpg,60 http://images.cocodataset.org/val2017/00...,"['horse', 'beach', 'sky', 'sand', 'horse', 'oc...","['brown', 'brown', 'blue', 'brown', 'brown', '...",A couple of jockey's riding the horses through...,A couple of jockey's riding the horses through...
2910,"581 189752\nName: id, dtype: int64",000000189752.jpg,581 http://images.cocodataset.org/val2017/0...,"['table', 'plate', 'glass', 'plate', 'hand', '...","['white', 'white', 'clear', 'white', 'white', ...","Pizza, salad, and a glass of Hefevwsen beer","Pizza, salad, and a glass of Hefevwsen beer"
22817,"4561 545958\nName: id, dtype: int64",000000545958.jpg,4561 http://images.cocodataset.org/val2017/...,"['building', 'wall', 'ground', 'box', 'balcony...","['blue', 'white', 'brown', 'black', 'blue', 'w...",A cow standing outside of a white building wit...,A cow standing outside of a white building wit...
18438,"3685 527528\nName: id, dtype: int64",000000527528.jpg,3685 http://images.cocodataset.org/val2017/...,"['grass', 'sky', 'cloud', 'kite', 'kite', 'sky...","['green', 'blue', 'white', 'blue', 'red', 'blu...",A group of kites are being flown in the air.,A group of kites are being flown in the air.


In [ ]:
check_aug = check_aug.drop(["file", "url", "objects", "attributes"], axis=1)

In [ ]:
check_aug.shape

(25014, 3)

In [ ]:
check_aug.to_csv(caption_dir + "MSCOCO/coco_check_modified.csv", index=False)